In [1]:
using GLMakie
using JLD

In [2]:
function get_metadata(file_name::String)
    c = jldopen(file_name, "r") do file
        return read(file, "metadata")
    end
end

get_metadata (generic function with 1 method)

In [3]:
function load_matrix(timestep, file_name)
    data_name = "zeta_$timestep"
    c = jldopen(file_name, "r") do file
        return read(file, data_name)
    end
end

load_matrix (generic function with 1 method)

In [6]:
function animation()
    simulation_name = "test7"
    file_name = "../../data/$simulation_name.jld"
    
    metadata = get_metadata(file_name)
    println(metadata)

    matrix = load_matrix(0, file_name)
    zeta_grid = matrix[:,:,1]

    grids = Observable(zeta_grid)

    # plot_time = Observable(0.0)

    fig = Figure()
    ax = Axis(fig[1, 1])
    heatmap!(ax, grids)

    T = floor(Int, metadata["T"])
    dt = metadata["dt"]
    sample_timestep = metadata["sample_timestep"]
    sample_interval = metadata["sample_interval"]

    for (timestep, time) in enumerate(0:dt:T)
        if timestep % (sample_timestep) == 0
            matrix = load_matrix(timestep, file_name)
            # println("$file_name, $timestep, | ", matrix[10,10,1])
            zeta_grid = matrix[:,:,1]

            grids[] = zeta_grid
            # plot_time[] = timestep            

            display(fig)

            # Set the matrix to nothing so the memoery can be freed up later by the garbage collector.
            matrix = nothing
            
            # Force garbage collection to clean up the memory used by the old array.
            GC.gc()
            sleep(0.2)
        end
    end
end

animation()

Dict{String, Real}("T" => 1.5768e7, "sample_timestep" => 48, "dt" => 1800.0, "sample_interval" => 86400.0)
